In [2]:
import pandas as pd
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
from sklearn import svm
import matplotlib.pyplot as plt
from Haldane_anis_supervise_class import *
from partial import *

In [ ]:
L = 51  # number of particles must be odd
keep = [0,1,2] # keep the part that we wanna do partial trace over that
bond = 50  # reasonable amount
p = partial(L, keep, bond)

In [3]:
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix()
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test)

Computing partial density matrices started .....
Tracing over training set (187 items)...


Tracing train set: 100%|██████████| 187/187 [00:01<00:00, 180.34it/s]


Tracing over test set (1600 items)...


Tracing test set: 100%|██████████| 1600/1600 [00:00<00:00, 4899.18it/s]


Partial density matrices computed in 1.90 seconds.
Computing Gram matrix for training set...


Gram Partial Train: 100%|██████████| 34969/34969 [00:00<00:00, 187592.64it/s]


Gram matrix for training set computed in 0.19 seconds.
Computing Gram matrix for testing set...


Gram Partial Test: 100%|██████████| 1600/1600 [00:03<00:00, 517.19it/s]

Gram matrix for testing set computed in 3.10 seconds.
